# Fine-tuning mBERT on CodiEsp-P

In this notebook, following a multi-label sequence classification approach, the mBERT model is fine-tuned on both the training and development sets of the CodiEsp-P corpus. Additionally, the predictions made by the model on the test set are saved, in order to futher evaluate the clinical coding performance of the model (see `results/CodiEsp-P/Evaluation.ipynb`).

In [1]:
import tensorflow as tf

# Auxiliary components
import sys
sys.path.append("..")
from nlp_utils import *

# mBERT tokenizer
from keras_bert import load_vocabulary, Tokenizer
model_path = "multi_cased_L-12_H-768_A-12/"
config_path = model_path + "bert_config.json"
checkpoint_path = model_path + "bert_model.ckpt"
vocab_file = "vocab.txt"
tokenizer = Tokenizer(token_dict=load_vocabulary(model_path + vocab_file), pad_index=0, cased=True)

# Hyper-parameters
text_col = "raw_text"
training = False
trainable = True
SEQ_LEN = 128
BATCH_SIZE = 16
EPOCHS = 30
LR = 3e-5

random_seed = 0
tf.set_random_seed(random_seed)

Using TensorFlow backend.


## Load text

Firstly, all text files from training and development CodiEsp corpora are loaded in different dataframes.

Also, CIE-Procedimiento codes are loaded.

In [2]:
corpus_path = "../datasets/codiesp_v4/"

### Training corpus

In [3]:
%%time
train_path = corpus_path + "train/text_files/"
train_files = [f for f in os.listdir(train_path) if os.path.isfile(train_path + f)]
train_data = load_text_files(train_files, train_path)
df_text_train = pd.DataFrame({'doc_id': [s.split('.txt')[0] for s in train_files], 'raw_text': train_data})

CPU times: user 5.81 ms, sys: 8.19 ms, total: 14 ms
Wall time: 13.9 ms


In [4]:
df_text_train.shape

(500, 2)

In [5]:
df_text_train.head()

,doc_id,raw_text
0,S0004-06142007000600016-2,Paciente varón de 35 años con tumoración en po...
1,S1137-66272009000500017-1,Lactante de sexo femenino que ingresó a los 7 ...
2,S0365-66912007001100010-1,Paciente de 63 años que refería déficit de agu...
3,S0365-66912009000300010-1,Se presenta el caso de un varón de 24 años de ...
4,S0211-69952013000500035-1,Se presenta el caso de un varón de 64 años sin...


In [6]:
df_text_train.raw_text[0]

'Paciente varón de 35 años con tumoración en polo superior de teste derecho hallada de manera casual durante una autoexploración, motivo por el cual acude a consulta de urología donde se realiza exploración física, apreciando masa de 1cm aproximado de diámetro dependiente de epidídimo, y ecografía testicular, que se informa como lesión nodular sólida en cabeza de epidídimo derecho. Se realiza RMN. Confirmando masa nodular, siendo el tumor adenomatoide de epidídimo la primera posibilidad diagnóstica.\n\nSe decide, en los dos casos, resección quirúrgica de tumoración nodular en cola epidídimo derecho, sin realización de orquiectomía posterior.\nEn ambos casos se realizó examen anátomopatológico de la pieza quirúrgica. Hallazgos histológicos macroscópicos: formación nodular de 1,5 cms (caso1) y 1,2 cms (caso 2) de consistencia firme, coloración blanquecina y bien delimitada. Microscópicamente se observa proliferación tumoral constituida por estructuras tubulares en las que la celularidad 

We also load the CIE-Procedimiento codes table:

In [7]:
df_codes_train = pd.read_table(corpus_path + "train/trainP.tsv", sep='\t', header=None)

In [8]:
df_codes_train.columns = ["doc_id", "code"]

In [9]:
df_codes_train.shape

(1550, 2)

In [10]:
df_codes_train.head()

,doc_id,code
0,S0004-06142005000700014-1,bw03zzz
1,S0004-06142005000700014-1,3e02329
2,S0004-06142005000700014-1,bw40zzz
3,S0004-06142005000700014-1,bv44zzz
4,S0004-06142005000700014-1,bn20


In [11]:
len(set(df_codes_train["doc_id"]))

435

### Development corpus

In [12]:
%%time
dev_path = corpus_path + "dev/text_files/"
dev_files = [f for f in os.listdir(dev_path) if os.path.isfile(dev_path + f)]
dev_data = load_text_files(dev_files, dev_path)
df_text_dev = pd.DataFrame({'doc_id': [s.split('.txt')[0] for s in dev_files], 'raw_text': dev_data})

CPU times: user 19.7 ms, sys: 4.05 ms, total: 23.8 ms
Wall time: 23.3 ms


In [13]:
df_text_dev.shape

(250, 2)

In [14]:
df_text_dev.head()

,doc_id,raw_text
0,S1698-44472004000100009-1,Varón de 64 años de edad con tumefacción mandi...
1,S1139-76322015000300013-1,Niña de tres años que acude a Urgencias tras l...
2,S1130-05582015000100004-1,Se presenta el caso de una mujer de 60 años de...
3,S1887-85712015000200005-1,Paciente varón de cinco años de edad que tras ...
4,S1699-65852010000300002-1,"LTR. Paciente de sexo masculino, de 32 años de..."


In [15]:
df_text_dev.raw_text[0]

'Varón de 64 años de edad con tumefacción mandibular derecha de 6 meses de evolución. La radiografía simple mostraba una lesión expansiva bien delimitada, osteolítica, multiloculada, localizada en rama horizontal mandibular. La tomografía computerizada presentaba una lesión expansiva con destrucción de la cortical ósea. Con el diagnóstico provisional de probable ameloblastoma se procedió a la resección-biopsia de la lesión. Mediante incisión interpapilar se expuso la mandíbula que mostraba la superficie abombada y destruída por una tumoración carnosa de consistencia densa que rodeaba la rama del nervio dentario inferior. Tras un cuidadoso curetaje de la cavidad ósea se reconstruyó la mandíbula y se repuso la mucosa. No hubo complicaciones postquirúrgicas.\n\nEl material remitido a Anatomía Patológica consistía en fragmentos tumorales de unos 2x1.5 cm, blanco-grisáceos al corte y de consistencia firme. Se tomaron diversas muestras que tras fijarse en formaldehído se incluyeron en parafi

We also load the CIE-Procedimiento codes table:

In [16]:
df_codes_dev = pd.read_table(corpus_path + "dev/devP.tsv", sep='\t', header=None)

In [17]:
df_codes_dev.columns = ["doc_id", "code"]

In [18]:
df_codes_dev.shape

(817, 2)

In [19]:
df_codes_dev.head()

,doc_id,code
0,S0004-06142005000900016-1,bt41zzz
1,S0004-06142005000900016-1,ct13
2,S0004-06142005001000011-1,3e1m39z
3,S0004-06142005001000011-1,0tcb
4,S0004-06142005001000011-1,bt02


In [20]:
len(set(df_codes_dev["doc_id"]))

222

We join the training and development CodiEsp codes dataframes together:

In [21]:
df_codes_train_dev = pd.concat([df_codes_train, df_codes_dev])

In [22]:
df_codes_train_dev.shape

(2367, 2)

In [23]:
df_codes_train_dev.head()

,doc_id,code
0,S0004-06142005000700014-1,bw03zzz
1,S0004-06142005000700014-1,3e02329
2,S0004-06142005000700014-1,bw40zzz
3,S0004-06142005000700014-1,bv44zzz
4,S0004-06142005000700014-1,bn20


## Creating corpora of annotated sentences

Leveraging the information available for the named-entity-recognition and normalization (NER-N) CodiEsp-X task, we create both a training and a development corpus of annotated sentences with CIE-Procedimiento codes.

Firstly, we pre-process the NER-N precedure-codes annotations available for both the training and development corpora.

In [24]:
# Training corpus

In [25]:
%%time

codiesp_x_train = pd.read_table(corpus_path + "train/trainX.tsv", sep='\t', header=None)

CPU times: user 42.5 ms, sys: 0 ns, total: 42.5 ms
Wall time: 41.9 ms


In [26]:
codiesp_x_train.columns = ["doc_id", "type", "code", "word", "location"]

In [27]:
codiesp_x_train.shape

(9181, 5)

In [28]:
codiesp_x_train.head()

,doc_id,type,code,word,location
0,S0004-06142005000700014-1,PROCEDIMIENTO,bw03zzz,Rx tórax,2163 2171
1,S0004-06142005000700014-1,PROCEDIMIENTO,3e02329,Estreptomicina intramuscular,2787 2801;2810 2823
2,S0004-06142005000700014-1,DIAGNOSTICO,n44.8,teste derecho aumentado de tamaño,1343 1376
3,S0004-06142005000700014-1,DIAGNOSTICO,z20.818,exposición a Brucella,594 615
4,S0004-06142005000700014-1,DIAGNOSTICO,r60.9,edemas,1250 1256


In [29]:
codiesp_x_train = codiesp_x_train[codiesp_x_train["type"] == "PROCEDIMIENTO"]

In [30]:
codiesp_x_train.shape

(1972, 5)

In [31]:
df_codes_train_ner = process_ner_labels(codiesp_x_train).sort_values(["doc_id", "start", "end"])

In [32]:
df_codes_train_ner.head()

,doc_id,type,code,word,start,end
0,S0004-06142005000700014-1,PROCEDIMIENTO,bw03zzz,Rx tórax,2163,2171
3,S0004-06142005000700014-1,PROCEDIMIENTO,bw40zzz,Ecografía abdominal,2173,2192
5,S0004-06142005000700014-1,PROCEDIMIENTO,bn20,TAC craneal,2194,2205
4,S0004-06142005000700014-1,PROCEDIMIENTO,bv44zzz,Ecografía testicular,2287,2307
1,S0004-06142005000700014-1,PROCEDIMIENTO,3e02329,Estreptomicina intramuscular,2787,2801


In [33]:
df_codes_train_ner.shape

(2769, 6)

In [34]:
# Development corpus

In [35]:
%%time

codiesp_x_dev = pd.read_table(corpus_path + "dev/devX.tsv", sep='\t', header=None)

CPU times: user 21.5 ms, sys: 12.1 ms, total: 33.5 ms
Wall time: 33 ms


In [36]:
codiesp_x_dev.columns = ["doc_id", "type", "code", "word", "location"]

In [37]:
codiesp_x_dev.shape

(4477, 5)

In [38]:
codiesp_x_dev.head()

,doc_id,type,code,word,location
0,S0004-06142005000900016-1,PROCEDIMIENTO,bt41zzz,ecografía renal derecha,307 316;348 361
1,S0004-06142005000900016-1,PROCEDIMIENTO,ct13,gammagrafía renal,739 756
2,S0004-06142005000900016-1,DIAGNOSTICO,q62.11,estenosis en la unión pieloureteral derecha,540 583
3,S0004-06142005000900016-1,DIAGNOSTICO,n28.89,ectasia pielocalicial,326 347
4,S0004-06142005000900016-1,DIAGNOSTICO,n39.0,infecciones del tracto urinario,198 229


In [39]:
codiesp_x_dev = codiesp_x_dev[codiesp_x_dev["type"] == "PROCEDIMIENTO"]

In [40]:
codiesp_x_dev.shape

(1046, 5)

In [41]:
df_codes_dev_ner = process_ner_labels(codiesp_x_dev).sort_values(["doc_id", "start", "end"])

In [42]:
df_codes_dev_ner.head()

,doc_id,type,code,word,start,end
0,S0004-06142005000900016-1,PROCEDIMIENTO,bt41zzz,ecografía renal derecha,307,316
1,S0004-06142005000900016-1,PROCEDIMIENTO,bt41zzz,ecografía renal derecha,348,361
2,S0004-06142005000900016-1,PROCEDIMIENTO,ct13,gammagrafía renal,739,756
3,S0004-06142005001000011-1,PROCEDIMIENTO,3e1m39z,diálisis peritoneal,95,114
7,S0004-06142005001000011-1,PROCEDIMIENTO,0270,angioplastia transluminal de la coronaria derecha,424,473


In [43]:
df_codes_dev_ner.shape

(1540, 6)

Now, using the character start-end positions of each sentence from the CodiEsp corpus (see `datasets/CodiEsp-Sentence-Split.ipynb`), we annotate the sentences with CIE-Procedimiento codes. Also, using mBERT tokenizer, each sentence is converted into a sequence of subwords, which are further converted into vocabulary indices (input IDs) and segments arrays (BERT input tensors). We also generate a *fragments* dataset indicating the number of produced annotated sentences for each document.

In [44]:
# Sentence-Split information
ss_corpus_path = "../datasets/CodiEsp-SSplit-text/"

### Training corpus

In [45]:
label_list = list(df_codes_train_dev["code"])

In [46]:
len(label_list)

2367

In [47]:
len(set(label_list))

727

In [48]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb_encoder = MultiLabelBinarizer()
mlb_encoder.fit([label_list])

MultiLabelBinarizer(classes=None, sparse_output=False)

In [49]:
# Number of distinct codes
num_labels = len(mlb_encoder.classes_)

In [50]:
num_labels

727

Only training texts that are annotated with CIE-Procedimiento codes are considered:

In [51]:
# Some train documents (texts) are not annotated 
len(set(df_text_train["doc_id"]) - set(df_codes_train_ner["doc_id"]))

65

In [52]:
train_doc_list = sorted(set(df_codes_train_ner["doc_id"]))

In [53]:
len(train_doc_list)

435

In [54]:
# Sentence-Split data

In [55]:
%%time
ss_sub_corpus_path = ss_corpus_path + "train/"
ss_files = [f for f in os.listdir(ss_sub_corpus_path) if os.path.isfile(ss_sub_corpus_path + f)]
ss_dict_train = load_ss_files(ss_files, ss_sub_corpus_path)

CPU times: user 11.6 ms, sys: 8.19 ms, total: 19.8 ms
Wall time: 19.2 ms


In [56]:
%%time
train_ind, train_seg, train_y, train_frag, train_start_end_frag = ss_create_frag_input_data_bert(df_text=df_text_train, 
                                                  text_col=text_col, 
                                                  df_ann=df_codes_train_ner, doc_list=train_doc_list, ss_dict=ss_dict_train,
                                                  tokenizer=tokenizer, lab_encoder=mlb_encoder, seq_len=SEQ_LEN,
                                                  greedy=False)

100%|██████████| 435/435 [00:06<00:00, 66.21it/s]


CPU times: user 6.67 s, sys: 35.7 ms, total: 6.7 s
Wall time: 6.65 s


In [57]:
# Sanity check

In [58]:
train_ind.shape

(7025, 128)

In [59]:
train_seg.shape

(7025, 128)

In [60]:
train_y.shape

(7025, 727)

In [61]:
len(train_frag)

435

In [62]:
len(train_start_end_frag)

7025

In [63]:
# Check n_frag distribution across texts
pd.Series(train_frag).describe()

count    435.000000
mean      16.149425
std        7.778513
min        4.000000
25%       10.500000
50%       15.000000
75%       20.000000
max       54.000000
dtype: float64

In [64]:
# Inspect a randomly selected text and its encoded version
check_id = np.random.randint(low=0, high=len(train_doc_list), size=1)[0]

In [65]:
check_id

302

In [66]:
train_doc_list[check_id]

'S1130-05582015000400005-1'

In [67]:
df_text_train[df_text_train["doc_id"] == train_doc_list[check_id]][text_col].values[0]

'Paciente H.V.P.D., 16 años, se presentó con queja principal de dificultad de fonación. Durante el examen físico, se observó que el paciente presentaba elevada estatura, patrón facial III, con severa hipoplasia maxilar, apiñamientos dentarios, relación de clase III de Angle, mordida abierta anterior, úvula bífida y una macroglosia. En la recolección de datos durante la anamnesis, fue revelado también que, al nacimiento, el paciente tenía un defecto en la pared abdominal que fue corregido posteriormente. Todos esos hallazgos contribuyeron para la formación del diagnóstico del síndrome de Beckwith-Wiedmann.\n\nEl paciente fue incorporado al tratamiento ortoquirúrgico, siendo observados problemas de deglución, fonación y dificultades respiratorias, con fuerte correlación con la macroglosia. Por consiguiente, se planeó para ese caso un procedimiento quirúrgico de glosectomía parcial, anterior al procedimiento de cirugía ortognática. La técnica utilizada fue la propuesta por Obwegeser et al

In [68]:
check_id_frag = sum(train_frag[:check_id])
for i in range(check_id_frag, check_id_frag + train_frag[check_id]):
    print(mlb_encoder.inverse_transform(np.array([train_y[i]])), "\n")

[()] 

[()] 

[()] 

[()] 

[()] 

[('0cb7',)] 

[()] 

[()] 

[('0bh1',)] 

[()] 

[()] 

[()] 

[()] 

[()] 

[()] 

[()] 

[()] 

[()] 



In [69]:
for i in range(check_id_frag, check_id_frag + train_frag[check_id]):
    print(list(zip([tokenizer._token_dict_inv[ind] for ind in train_ind[i]][1:len(train_start_end_frag[i])+1], 
               train_start_end_frag[i])))
    print("\n")

[('Pac', (0, 3)), ('##iente', (3, 8)), ('H', (9, 10)), ('.', (10, 11)), ('V', (11, 12)), ('.', (12, 13)), ('P', (13, 14)), ('.', (14, 15)), ('D', (15, 16)), ('.', (16, 17)), (',', (17, 18)), ('16', (19, 21)), ('años', (22, 26)), (',', (26, 27)), ('se', (28, 30)), ('presentó', (31, 39)), ('con', (40, 43)), ('que', (44, 47)), ('##ja', (47, 49)), ('principal', (50, 59)), ('de', (60, 62)), ('di', (63, 65)), ('##ficu', (65, 69)), ('##ltad', (69, 73)), ('de', (74, 76)), ('fon', (77, 80)), ('##ación', (80, 85)), ('.', (85, 86))]


[('Durante', (87, 94)), ('el', (95, 97)), ('examen', (98, 104)), ('físico', (105, 111)), (',', (111, 112)), ('se', (113, 115)), ('ob', (116, 118)), ('##ser', (118, 121)), ('##vó', (121, 123)), ('que', (124, 127)), ('el', (128, 130)), ('paciente', (131, 139)), ('presenta', (140, 148)), ('##ba', (148, 150)), ('elevada', (151, 158)), ('estat', (159, 164)), ('##ura', (164, 167)), (',', (167, 168)), ('patrón', (169, 175)), ('fac', (176, 179)), ('##ial', (179, 182)), ('II

In [70]:
check_id_frag = sum(train_frag[:check_id])
for frag in train_ind[check_id_frag:check_id_frag + train_frag[check_id]]:
    print(' '.join([tokenizer._token_dict_inv[ind] for ind in frag]), "\n")

[CLS] Pac ##iente H . V . P . D . , 16 años , se presentó con que ##ja principal de di ##ficu ##ltad de fon ##ación . [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 

[CLS] Durante el examen físico , se ob ##ser ##vó que el paciente presenta ##ba elevada estat ##ura , patrón fac ##ial III , con sever ##a hip ##op ##lasi ##a maxi ##lar , api ##ña ##mientos den ##tario ##s , relación de clase III de Angle , mor ##dida abierta anterior , ú ##vul ##a bí

In [71]:
# Fragment labels distribution
pd.Series(np.sum(train_y, axis=1)).describe()

count    7025.000000
mean        0.304911
std         0.652322
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         6.000000
dtype: float64

### Development corpus

Only development texts that are annotated with CIE-Procedimiento codes are considered:

In [72]:
# Some dev documents (texts) are not annotated 
len(set(df_text_dev["doc_id"]) - set(df_codes_dev_ner["doc_id"]))

28

In [73]:
dev_doc_list = sorted(set(df_codes_dev_ner["doc_id"]))

In [74]:
len(dev_doc_list)

222

In [75]:
%%time
ss_sub_corpus_path = ss_corpus_path + "dev/"
ss_files = [f for f in os.listdir(ss_sub_corpus_path) if os.path.isfile(ss_sub_corpus_path + f)]
ss_dict_dev = load_ss_files(ss_files, ss_sub_corpus_path)

CPU times: user 16.8 ms, sys: 19 µs, total: 16.8 ms
Wall time: 16.5 ms


In [76]:
%%time
dev_ind, dev_seg, dev_y, dev_frag, dev_start_end_frag = ss_create_frag_input_data_bert(df_text=df_text_dev, 
                                                  text_col=text_col, 
                                                  df_ann=df_codes_dev_ner, doc_list=dev_doc_list, ss_dict=ss_dict_dev,
                                                  tokenizer=tokenizer, lab_encoder=mlb_encoder, seq_len=SEQ_LEN,
                                                  greedy=False)

100%|██████████| 222/222 [00:03<00:00, 63.11it/s]

CPU times: user 3.57 s, sys: 11.5 ms, total: 3.59 s
Wall time: 3.56 s


In [77]:
# Sanity check

In [78]:
dev_ind.shape

(3808, 128)

In [79]:
dev_seg.shape

(3808, 128)

In [80]:
dev_y.shape

(3808, 727)

In [81]:
len(dev_frag)

222

In [82]:
len(dev_start_end_frag)

3808

In [83]:
# Check n_frag distribution across texts
pd.Series(dev_frag).describe()

count    222.000000
mean      17.153153
std        8.327785
min        4.000000
25%       11.000000
50%       15.000000
75%       21.000000
max       65.000000
dtype: float64

In [84]:
# Inspect a randomly selected text and its encoded version
check_id = np.random.randint(low=0, high=len(dev_doc_list), size=1)[0]

In [85]:
check_id

99

In [86]:
dev_doc_list[check_id]

'S0365-66912011001000005-1'

In [87]:
df_text_dev[df_text_dev["doc_id"] == dev_doc_list[check_id]][text_col].values[0]

'Mujer de 52 años que refirió disminución de agudeza visual (AV) espontánea en su ojo izquierdo (OI) ambliope. No tenía el antecedente de parto prematuro, ni historia familiar de enfermedad ocular.\nEn la primera exploración, la mejor AV corregida fue de 1 en el ojo derecho (OD) (-1 cilindro 150o +3 esfera; longitud axial de 22,5mm) y de 0,025 en el ojo izquierdo (OI) (-0,75 cilindro 25o -13,25 esfera; longitud axial de 25,3mm).\nEl estudio biomicroscópico del OI permitió identificar la mancha de Mittendorf localizada en el cuadrante nasal inferior de la cápsula posterior del cristalino y el extremo anterior de la AHP con contenido hemático en su interior. El ojo adelfo era normal. Los diámetros corneales horizontales midieron 12mm en ambos ojos (AO) y la tonometría de aplanación fue de 18mm Hg en AO.\n\nLa oftalmoscopia indirecta del OI mostró una hemorragia intravítrea leve difusa. El trayecto de la AHP se pudo seguir desde el nervio óptico hasta la cristaloides posterior. La arteria

In [88]:
check_id_frag = sum(dev_frag[:check_id])
for i in range(check_id_frag, check_id_frag + dev_frag[check_id]):
    print(mlb_encoder.inverse_transform(np.array([dev_y[i]])), "\n")

[()] 

[()] 

[()] 

[()] 

[()] 

[()] 

[()] 

[()] 

[()] 

[()] 

[()] 

[()] 

[()] 

[()] 

[()] 

[()] 

[()] 

[('b34vzzz',)] 

[()] 

[()] 

[()] 



In [89]:
for i in range(check_id_frag, check_id_frag + dev_frag[check_id]):
    print(list(zip([tokenizer._token_dict_inv[ind] for ind in dev_ind[i]][1:len(dev_start_end_frag[i])+1], 
               dev_start_end_frag[i])))
    print("\n")

[('Mujer', (0, 5)), ('de', (6, 8)), ('52', (9, 11)), ('años', (12, 16)), ('que', (17, 20)), ('ref', (21, 24)), ('##iri', (24, 27)), ('##ó', (27, 28)), ('dis', (29, 32)), ('##minución', (32, 40)), ('de', (41, 43)), ('ag', (44, 46)), ('##ude', (46, 49)), ('##za', (49, 51)), ('visual', (52, 58)), ('(', (59, 60)), ('AV', (60, 62)), (')', (62, 63)), ('es', (64, 66)), ('##pont', (66, 70)), ('##án', (70, 72)), ('##ea', (72, 74)), ('en', (75, 77)), ('su', (78, 80)), ('ojo', (81, 84)), ('izquierdo', (85, 94)), ('(', (95, 96)), ('O', (96, 97)), ('##I', (97, 98)), (')', (98, 99)), ('amb', (100, 103)), ('##lio', (103, 106)), ('##pe', (106, 108)), ('.', (108, 109))]


[('No', (110, 112)), ('tenía', (113, 118)), ('el', (119, 121)), ('ante', (122, 126)), ('##cedent', (126, 132)), ('##e', (132, 133)), ('de', (134, 136)), ('parto', (137, 142)), ('prema', (143, 148)), ('##tur', (148, 151)), ('##o', (151, 152)), (',', (152, 153)), ('ni', (154, 156)), ('historia', (157, 165)), ('familiar', (166, 174)), ('

In [90]:
check_id_frag = sum(dev_frag[:check_id])
for frag in dev_ind[check_id_frag:check_id_frag + dev_frag[check_id]]:
    print(' '.join([tokenizer._token_dict_inv[ind] for ind in frag]), "\n")

[CLS] Mujer de 52 años que ref ##iri ##ó dis ##minución de ag ##ude ##za visual ( AV ) es ##pont ##án ##ea en su ojo izquierdo ( O ##I ) amb ##lio ##pe . [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 

[CLS] No tenía el ante ##cedent ##e de parto prema ##tur ##o , ni historia familiar de enfermedad o ##cular . [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [P

In [91]:
# Fragment labels distribution
pd.Series(np.sum(dev_y, axis=1)).describe()

count    3808.000000
mean        0.309086
std         0.637504
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         5.000000
dtype: float64

### Training & Development corpus

We merge the previously generated datasets:

In [92]:
# Indices
train_dev_ind = np.concatenate((train_ind, dev_ind))

In [93]:
train_dev_ind.shape

(10833, 128)

In [94]:
# Segments
train_dev_seg = np.concatenate((train_seg, dev_seg))

In [95]:
train_dev_seg.shape

(10833, 128)

In [96]:
# y
train_dev_y = np.concatenate((train_y, dev_y))

In [97]:
train_dev_y.shape

(10833, 727)

## Fine-tuning

Using the corpus of labeled sentences, we fine-tune the model on a multi-label sentence classification task.

In [98]:
from keras.backend.tensorflow_backend import set_session

# Prevent GPU memory allocation problems
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

In [99]:
from keras_bert import load_trained_model_from_checkpoint

model = load_trained_model_from_checkpoint(
    config_file=config_path, 
    checkpoint_file=checkpoint_path, 
    training=training,                                       
    trainable=trainable, 
    seq_len=SEQ_LEN
)

In [100]:
model.inputs

[<tf.Tensor 'Input-Token:0' shape=(?, 128) dtype=float32>,
 <tf.Tensor 'Input-Segment:0' shape=(?, 128) dtype=float32>]

In [101]:
model.outputs

[<tf.Tensor 'Encoder-12-FeedForward-Norm/add_1:0' shape=(?, 128, 768) dtype=float32>]

In [102]:
from keras.layers import Dense, Activation
from keras.models import Model
from keras.initializers import glorot_uniform
from keras_bert.layers import Extract

dense_cls = Extract(index=0, name='Extract')(model.output) # In order to extract CLS token embedding
dense_out = Dense(units=num_labels, kernel_initializer=glorot_uniform(seed=random_seed))(dense_cls) # Multi-label classification
outputs = Activation('sigmoid')(dense_out)

model = Model(model.inputs, outputs)

In [103]:
model.inputs

[<tf.Tensor 'Input-Token:0' shape=(?, 128) dtype=float32>,
 <tf.Tensor 'Input-Segment:0' shape=(?, 128) dtype=float32>]

In [104]:
model.outputs

[<tf.Tensor 'activation_1/Sigmoid:0' shape=(?, 727) dtype=float32>]

In [ ]:
%%time
from keras_radam import RAdam

model.compile(
    optimizer=RAdam(learning_rate=LR),
    loss='binary_crossentropy'
)

history = model.fit(
    x=[train_dev_ind, train_dev_seg],
    y=train_dev_y,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    shuffle=True
)

Epoch 1/30
10833/10833 [==============================] - 161s 15ms/step - loss: 0.2331
Epoch 2/30
10833/10833 [==============================] - 160s 15ms/step - loss: 0.0251
Epoch 3/30
10833/10833 [==============================] - 161s 15ms/step - loss: 0.0085
Epoch 4/30
10833/10833 [==============================] - 161s 15ms/step - loss: 0.0052
Epoch 5/30
10833/10833 [==============================] - 161s 15ms/step - loss: 0.0041
Epoch 6/30
10833/10833 [==============================] - 161s 15ms/step - loss: 0.0037
Epoch 7/30
 9584/10833 [=========================>....] - ETA: 18s - loss: 0.0035

## Test set predictions

Finally, the predictions made by the model on the test set are saved. For this purpose, firstly, each sentence from the test corpus must be converted into a sequence of subwords (input IDs and attention mask arrays). Then, the predictions made by the model at the sentence-level are saved, to be further evaluated at document-level (see `results/CodiEsp-P/Evaluation.ipynb`).

In [108]:
%%time
test_path = corpus_path + "test/text_files/"
test_files = [f for f in os.listdir(test_path) if os.path.isfile(test_path + f)]
test_data = load_text_files(test_files, test_path)
df_text_test = pd.DataFrame({'doc_id': [s.split('.txt')[0] for s in test_files], 'raw_text': test_data})

CPU times: user 5.75 ms, sys: 336 µs, total: 6.09 ms
Wall time: 5.17 ms


In [109]:
df_text_test.shape

(250, 2)

In [110]:
df_text_test.head()

,doc_id,raw_text
0,S1698-44472004000400012-1,"Varón de 54 años de edad, remitido a nuestro s..."
1,S1130-05582012000300005-1,Acude a nuestras consultas a un paciente que p...
2,S0212-16112009000300015-1,Se trató de un varón de 77 años con antecedent...
3,S1139-76322014000500014-1,Niño de cinco años derivado por su pediatra de...
4,S0212-71992004000300009-1,Varón de 22 años de edad que acude a consultas...


In [111]:
df_text_test.raw_text[0]

'Varón de 54 años de edad, remitido a nuestro servicio en mayo del 2003 por presentar odontalgia en relación con un tercer molar inferior derecho erupcionado. A la inspección oral se pudo observar la existencia de una tumefacción que expandía las corticales vestibulo-linguales, en la región del tercer molar mandibular derecho, cariado por distal. La mucosa oral estaba indemne, y no se palpaban adenomegalias cervicales. El paciente refería la existencia de una hipoestesia en el territorio de distribución del nervio mentoniano de quince días de evolución. En la ortopantomografía, se evidenció la presencia de una imagen radiolúcida, de contornos poco definidos, en el cuerpo mandibular derecho. Dos días después, bajo anestesia local, se procedió a la exodoncia del tercer molar y curetaje-biopsia del tejido subyacente. Durante el acto operatorio, se produjo una intensa hemorragia, que pudo ser cohibida con el empleo de Surgicel (Johnson & Johnson, Nuevo Brunswick, NJ) y mediante el empaquet

In [112]:
test_doc_list = sorted(set(df_text_test["doc_id"]))

In [113]:
len(test_doc_list)

250

In [115]:
%%time
ss_sub_corpus_path = ss_corpus_path + "test/"
ss_files = [f for f in os.listdir(ss_sub_corpus_path) if os.path.isfile(ss_sub_corpus_path + f)]
ss_dict_test = load_ss_files(ss_files, ss_sub_corpus_path)

CPU times: user 9.24 ms, sys: 0 ns, total: 9.24 ms
Wall time: 8.4 ms


In [116]:
%%time
test_ind, test_seg, _, test_frag, _ = ss_create_frag_input_data_bert(df_text=df_text_test, 
                                                  text_col=text_col,
                                                  # Since labels are ignored, we pass df_codes_train_ner as df_ann
                                                  df_ann=df_codes_train_ner, doc_list=test_doc_list, ss_dict=ss_dict_test,
                                                  tokenizer=tokenizer, lab_encoder=mlb_encoder, seq_len=SEQ_LEN,
                                                  greedy=False)

100%|██████████| 250/250 [00:01<00:00, 171.24it/s]

CPU times: user 1.55 s, sys: 0 ns, total: 1.55 s
Wall time: 1.51 s


In [119]:
%%time
test_preds = model.predict([test_ind, test_seg])

CPU times: user 6.19 s, sys: 502 ms, total: 6.69 s
Wall time: 14 s


In [120]:
test_preds.shape

(3955, 727)

In [121]:
results_dir_path = "../results/CodiEsp-P/"

In [178]:
%%time
np.save(file=results_dir_path + "predictions/mbert_seed_" + str(random_seed) + "_test_preds.npy", arr=test_preds)

CPU times: user 2.21 ms, sys: 4.65 ms, total: 6.87 ms
Wall time: 6.02 ms


In [122]:
# To be further used when evaluating model performance at document level
np.save(file=results_dir_path + "mbert_test_frags.npy", arr=test_frag)